In [2]:
import gensim
import json
import re
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim 

morph = MorphAnalyzer()

C:\Users\ksenia\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Домашнее задание

Основаная задача - **построить хорошую тематическую модель с интерпретируемыми топиками**.

1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - `pip install pymorphy2[fast]`, можно использовать какой-то другой токенизатор); 

2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases); 

3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова); 

In [3]:
habr_texts = open('habr_texts.txt', encoding = 'utf-8').read().splitlines()

In [4]:
def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

def clean(words):
    clean = [morph.parse(word)[0].normal_form for word in words if len(set(word)) > 1]
    return clean

In [5]:
import collections

In [6]:
f = open('habr_texts.vw', 'w', encoding='utf-8')

for i, text in enumerate(habr_texts):
    c = collections.Counter(text)
    doc = 'doc_'+ str(i) + ' '
    vw_text = ' '.join([x+':'+str(c[x]) for x in c])
    
    f.write(doc + vw_text  + '\n')
f.close()



In [8]:
texts = [clean(word_tokenize(text.lower())) for text in habr_texts]

In [4]:
#texts[:10]

In [9]:
# для нграммов
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

Для моделей нужно сделать словарь.

In [10]:
dictinary = gensim.corpora.Dictionary(texts)

In [11]:
dictinary.filter_extremes(no_above=0.3, no_below=30)
dictinary.compactify()

In [12]:
print(dictinary)

Dictionary(7649 unique tokens: ['.net', '2-х', '3.0', 'address', 'api']...)


4) постройте несколько LDA моделей (переберите количество тем, можете поменять eta, alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем;

5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

In [13]:
corpus = [dictinary.doc2bow(text) for text in texts]


Перебираем несколько моделей. Выведем результаты последней, которые получились лучше всего

In [14]:
lda = gensim.models.LdaMulticore(corpus, 100, id2word=dictinary, passes=1) # если поддерживается многопоточность


C:\Users\ksenia\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [15]:
#lda.print_topics()

In [16]:
lda1 = gensim.models.LdaMulticore(corpus, 10, id2word=dictinary, passes=1) 

In [26]:
#lda1.print_topics()

In [17]:
lda2 = gensim.models.LdaMulticore(corpus, 5, id2word=dictinary, passes=3) 
#вот это выглядит неплохо

In [31]:
#lda2.print_topics()

In [18]:
lda3 = gensim.models.LdaMulticore(corpus, 7, id2word=dictinary, passes=10) 

In [23]:
#lda3.print_topics()

Третья модель оказалась самой удачной. Вот несколько хороших тем из нее.

In [24]:
lda3.print_topics()[2]

(2,
 '0.008*"значение" + 0.006*"язык" + 0.005*"модель" + 0.004*"элемент" + 0.004*"точка" + 0.004*"алгоритм" + 0.004*"текст" + 0.004*"изображение" + 0.004*"цикл" + 0.004*"объект"')

In [25]:
lda3.print_topics()[3]

(3,
 '0.019*"игра" + 0.007*"игрок" + 0.005*"массив" + 0.004*"технология" + 0.004*"машина" + 0.004*"доклад" + 0.004*"виртуальный" + 0.003*"подход" + 0.003*"режим" + 0.003*"облачный"')

In [27]:
lda3.print_topics()[4]

(4,
 '0.013*"устройство" + 0.008*"сеть" + 0.005*"сервер" + 0.005*"доступ" + 0.004*"безопасность" + 0.004*"управление" + 0.003*"технология" + 0.003*"правило" + 0.003*"связь" + 0.003*"рабочий"')

Мы, кажется, не убрали стоп-слова! Надо исправляться

In [28]:
import nltk
from nltk.corpus import stopwords

In [31]:
def stop_word(ttext):
    filtered_words = [word for word in ttext if word not in stopwords.words('russian')]
    return filtered_words

In [32]:
def all_stop(ttexts):
    return [stop_word(ttext) for ttext in ttexts]


In [33]:
cleaned = all_stop(texts)

In [34]:
dictinary2 = gensim.corpora.Dictionary(cleaned)

In [35]:
dictinary2.filter_extremes(no_above=0.3, no_below=45)
dictinary2.compactify()

In [36]:
print(dictinary2)

Dictionary(5651 unique tokens: ['.net', '2-х', '3.0', 'address', 'api']...)


In [40]:
corpus = [dictinary2.doc2bow(text) for text in cleaned]

In [41]:
lda4 = gensim.models.LdaMulticore(corpus, 10, id2word=dictinary2, passes=1) # если поддерживается многопоточность


In [43]:
#lda4.print_topics()

In [44]:
lda5 = gensim.models.LdaMulticore(corpus, 7, id2word=dictinary2, passes=5)
lda5.print_topics()

[(0,
  '0.007*"сеть" + 0.006*"сайт" + 0.004*"доступ" + 0.004*"безопасность" + 0.003*"технология" + 0.003*"услуга" + 0.003*"интернет" + 0.003*"точка" + 0.003*"клиент" + 0.003*"сервис"'),
 (1,
  '0.011*"игра" + 0.004*"игрок" + 0.003*"друг" + 0.003*"какой-то" + 0.003*"исследование" + 0.003*"что-то" + 0.002*"понять" + 0.002*"мозг" + 0.002*"продукт" + 0.002*"играть"'),
 (2,
  '0.062*"lt" + 0.061*"gt" + 0.012*"if" + 0.011*"объект" + 0.010*"public" + 0.008*"int" + 0.008*"string" + 0.008*"return" + 0.007*"класс" + 0.006*"метод"'),
 (3,
  '0.008*"файл" + 0.008*"устройство" + 0.005*"сеть" + 0.005*"сервер" + 0.004*"модуль" + 0.004*"программа" + 0.004*"машина" + 0.003*"режим" + 0.003*"канал" + 0.003*"память"'),
 (4,
  '0.013*"файл" + 0.009*"gt" + 0.007*"запрос" + 0.007*"сервер" + 0.007*"in" + 0.006*"from" + 0.005*"client" + 0.005*"строка" + 0.004*"ключ" + 0.004*"name"'),
 (5,
  '0.006*"устройство" + 0.006*"температура" + 0.005*"вакансия" + 0.004*"скорость" + 0.004*"датчик" + 0.004*"кампания" + 0.0

In [45]:
lda5.print_topics()[0]

(0,
 '0.007*"сеть" + 0.006*"сайт" + 0.004*"доступ" + 0.004*"безопасность" + 0.003*"технология" + 0.003*"услуга" + 0.003*"интернет" + 0.003*"точка" + 0.003*"клиент" + 0.003*"сервис"')

In [46]:
lda5.print_topics()[1]

(1,
 '0.011*"игра" + 0.004*"игрок" + 0.003*"друг" + 0.003*"какой-то" + 0.003*"исследование" + 0.003*"что-то" + 0.002*"понять" + 0.002*"мозг" + 0.002*"продукт" + 0.002*"играть"')

In [47]:
lda5.print_topics()[3]

(3,
 '0.008*"файл" + 0.008*"устройство" + 0.005*"сеть" + 0.005*"сервер" + 0.004*"модуль" + 0.004*"программа" + 0.004*"машина" + 0.003*"режим" + 0.003*"канал" + 0.003*"память"')

6) между словарем и обучением модели добавьте tfidf (gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]);

7) заново обучите LDA c теми же параметрами (параметрами самой лучшей модели, заново перебирать не нужно);

8) в отдельной ячейке опишите как изменилась модель (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем

10) в отдельной ячейки напечатайте таблицу с темами лучшей NMF модели, сравните их с теми, что получились в LDA.

In [48]:
from gensim.models import TfidfModel

In [50]:
gensim.models.TfidfModel(corpus, id2word=dictinary2); 

In [52]:
lda5 = gensim.models.LdaMulticore(corpus, 7, id2word=dictinary2, passes=1)
lda5.print_topics()

[(0,
  '0.040*"gt" + 0.031*"lt" + 0.006*"файл" + 0.006*"return" + 0.005*"result" + 0.005*"if" + 0.004*"function" + 0.004*"public" + 0.004*"+=" + 0.003*"amp"'),
 (1,
  '0.006*"gt" + 0.004*"игра" + 0.004*"lt" + 0.004*"файл" + 0.004*"сеть" + 0.003*"массив" + 0.002*"сайт" + 0.002*"устройство" + 0.002*"сервер" + 0.002*"объект"'),
 (2,
  '0.004*"устройство" + 0.003*"модуль" + 0.003*"игра" + 0.003*"игрок" + 0.002*"gt" + 0.002*"клиент" + 0.002*"группа" + 0.002*"сеть" + 0.002*"сервер" + 0.002*"сайт"'),
 (3,
  '0.019*"gt" + 0.009*"lt" + 0.004*"return" + 0.003*"result" + 0.003*"for" + 0.003*"игра" + 0.003*"устройство" + 0.002*"значение" + 0.002*"объект" + 0.002*"function"'),
 (4,
  '0.024*"gt" + 0.012*"lt" + 0.004*"сеть" + 0.004*"сайт" + 0.004*"return" + 0.003*"файл" + 0.003*"объект" + 0.003*"сервер" + 0.003*"значение" + 0.003*"сообщение"'),
 (5,
  '0.011*"lt" + 0.009*"gt" + 0.005*"игра" + 0.004*"файл" + 0.003*"result" + 0.003*"сеть" + 0.003*"игрок" + 0.002*"return" + 0.002*"элемент" + 0.002*"for

Кажется, что лучше не стало. В целом многие темы оказываются загрязненными терминами из языков программирования, лучше всего выделяются темы, связанные с современными технологиями, играми и устройством сайтов, и наборы слов для них 

In [53]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

In [54]:
stexts = [' '.join(text) for text in cleaned]

In [55]:
vectorizer = TfidfVectorizer(max_features=25000, min_df=5, max_df=0.3, lowercase=False)
X = vectorizer.fit_transform(stexts)

In [56]:
model = NMF(n_components=30)

In [57]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=30, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [58]:
def get_nmf_topics(model, n_top_words):
    
    #id слов.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(30):
        
        #топ n слов для темы.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [59]:
get_nmf_topics(model, 10) 

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 21,Topic # 22,Topic # 23,Topic # 24,Topic # 25,Topic # 26,Topic # 27,Topic # 28,Topic # 29,Topic # 30
0,продукт,gt,игра,космический,объект,сеть,звук,js,windows,файл,...,язык,товар,public,дата,бот,камера,российский,if,the,сайт
1,клиент,lt,игрок,спутник,значение,трафик,сигнал,react,microsoft,docker,...,программирование,скидка,string,центр,telegram,устройство,рынок,int,of,страница
2,сотрудник,div,игровой,орбита,элемент,связь,наушник,css,linux,сервер,...,php,магазин,void,сервер,сообщение,смартфон,страна,amp,to,реклама
3,бизнес,class,играть,ракета,метод,оператор,усилитель,javascript,studio,php,...,программист,цена,new,инфраструктура,телеграм,аккумулятор,россия,lt,in,браузер
4,менеджер,name,vr,марс,блок,ip,частота,angular,visual,sudo,...,лекция,распродажа,this,облачный,чат,датчик,налог,return,and,контент
5,crm,this,персонаж,аппарат,строка,интернет,звуковой,веб,net,http,...,java,покупатель,return,облако,bot,телефон,рубль,else,is,google
6,заказчик,props,steam,земля,алгоритм,dpi,музыка,vue,azure,nginx,...,перевод,пятница,private,услуга,api,видео,доход,null,on,домен
7,программист,html,unity,луна,класс,устройство,искажение,браузер,браузер,скрипт,...,программа,покупка,var,оборудование,мессенджер,экран,закон,char,for,рекламный
8,опыт,return,геймплей,наса,текстура,канал,акустический,компонент,обновление,etc,...,курс,чёрный,класс,цод,канал,корпус,деньга,void,from,com
9,тестирование,type,движок,полёт,точка,сетевой,напряжение,dom,server,usr,...,python,продавец,class,сервис,message,дисплей,бизнес,std,you,сервер


In [66]:
vectorizer = TfidfVectorizer(max_features=25000, min_df=6, max_df=0.2, lowercase=False)
X = vectorizer.fit_transform(stexts)

In [67]:
model = NMF(n_components=30)
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=30, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [68]:
get_nmf_topics(model, 10) 

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 21,Topic # 22,Topic # 23,Topic # 24,Topic # 25,Topic # 26,Topic # 27,Topic # 28,Topic # 29,Topic # 30
0,бизнес,lt,игра,марс,docker,страница,камера,ракета,android,js,...,бот,пациент,процессор,центр,public,звук,if,vr,php,робот
1,сотрудник,div,игрок,земля,http,тест,смартфон,спутник,google,react,...,telegram,мозг,диск,дата,string,сигнал,int,виртуальный,symfony,ребёнок
2,рынок,class,игровой,планета,sudo,запрос,видео,ступень,ios,css,...,сообщение,учёный,память,инфраструктура,void,наушник,amp,реальность,yii,автомобиль
3,деньга,name,играть,космический,etc,тестирование,регистратор,двигатель,мобильный,javascript,...,чат,клетка,intel,услуга,this,частота,return,oculus,laravel,lego
4,российский,props,персонаж,луна,nginx,текст,телефон,spacex,app,angular,...,телеграм,заболевание,ssd,облачный,new,усилитель,else,шлем,yii2,датчик
5,стартап,html,steam,аппарат,ip,строка,карта,космический,swift,веб,...,bot,исследование,ядро,оборудование,return,звуковой,null,vive,api,робототехника
6,страна,std,unity,орбита,com,блок,аккумулятор,пуск,play,vue,...,api,болезнь,производительность,облако,private,напряжение,char,машина,psr,машина
7,зарплата,this,геймплей,астероид,168,таблица,экран,орбита,view,браузер,...,канал,ген,накопитель,цод,класс,искажение,void,htc,go,движение
8,ит,type,движок,солнечный,amp,документ,дисплей,носитель,gradle,html,...,мессенджер,врач,гб,провайдер,class,ток,end,unreal,форум,беспилотный
9,программист,text,жанр,станция,usr,библиотека,xiaomi,корабль,firebase,компонент,...,message,лечение,компьютер,ит,var,музыка,const,вм,function,дрон


В целом кажется, что последняя модель дала наилучшие результаты. Темы выделяются гораздо лучше, чем в первых моделях.